In [16]:

######## prompting ONLY GPT########


import base64
from pathlib import Path
from openai import OpenAI
from tqdm import tqdm

API_KEY = os.getenv("OPENAI_API_KEY")
if API_KEY is None:
    raise EnvironmentError("OPENAI_API_KEY is not set in the environment variables")

client = OpenAI(api_key=API_KEY)


output_file = Path("/.../GPT_output_prompt_only.txt") 
output_file.parent.mkdir(parents=True, exist_ok=True) 

SEPARATOR = "\n\n\n——\n\n\n"

prompt = '''I have a dataset in an arbitrary language, but because its script is particularly difficult to format for computer input, different <token_N>, where N is the number, will represent different characters. For example, |<token_1><token_3><token_6>| can substitute for a string or sentence composed of three characters. Please use this as background knowledge for handling the following questions.

The Avoiuli writing system is used on Pentecost Island in Vanuatu, a Pacific Island Country in the Pacific region of Melanesia. With a design inspired by traditional sand drawings, Avoiuli was developed over fourteen years by Chief Viraleo Boborenvanua, as part of a movement he leads to revitalize Melanesian culture and language.
One of the languages that Avoiuli is used to write is Bislama, one of the official languages of Vanuatu. As you will see, Bislama is a creole language – many of its words come from English, some come from French, and others from the indigenous languages of Vanuatu.
Below is a sign written in Bislama, using the Avoiuli script, on the front of a college on Pentecost Island. For the purposes of this problem, each of the words in the script below has been allotted a number:

1. <token_1><token_2><token_3>
2. <token_1><token_4><token_5><token_6><token_2>
3. <token_3><token_7><token_8><token_9>
4. <token_6><token_10><token_11><token_6><token_11><token_6><token_1><token_5><token_11>
5. <token_5><token_2><token_11><token_1><token_11><token_5><token_12><token_13><token_12><token_14>
6. <token_3><token_7><token_8><token_9>
7. <token_6><token_15><token_16><token_15><token_14>
8. <token_1><token_2><token_4><token_5>
9. <token_17><token_15><token_7><token_8><token_1><token_8><token_17><token_15>
10. <token_11><token_16><token_8><token_13><token_8><token_5><token_18><token_12><token_6>
11. <token_8><token_14>
12. <token_11><token_6><token_11><token_5><token_2><token_14><token_10><token_11><token_19>
13. <token_7><token_15><token_1><token_2>
14. <token_20><token_15><token_7><token_15><token_16>
15. <token_7><token_8><token_7><token_8><token_20><token_15><token_5><token_15>

Q 6.1 On your answer sheet, give the number of a word (any one, if there are multiple) which corresponds to these Bislama words:

(a) filosofi 

(b) institiut

(c) blong

Q 6.2 Give the numbers of a word (any one, if there are multiple) which means:

(a) village 

(b) centre 

(c) teach

(d) technology

(e) humanity 

Q 6.3 Which geographical area of the Pacific is mentioned in the text?'''

image_path = None  


def image_to_data_url(path: Path) -> str:
    b64 = base64.b64encode(path.read_bytes()).decode("utf-8")
    ext = path.suffix.lstrip(".").lower() or "png"
    return f"data:image/{ext};base64,{b64}"

for i in tqdm(range(10), desc="Calling OpenAI API"):
    content = [{"type": "text", "text": prompt}]

    if image_path is not None:
        content.append({
            "type": "image_url",
            "image_url": {
                "url": image_to_data_url(image_path)
            }
        })

    response = client.chat.completions.create(
        model="gpt-5.2",
        messages=[
            {"role": "user", "content": content}
        ],
    )

    reply = response.choices[0].message.content or ""
    print(reply)
    if output_file.stat().st_size > 0:
        reply = SEPARATOR + reply

    with output_file.open("a", encoding="utf-8") as f:
        f.write(f"[run {i}]\n{reply}")



Calling OpenAI API:  50%|████████████▌            | 1/2 [00:07<00:07,  7.61s/it]

### Q 6.1  (give a word number)

To answer these, we match repeated Avoiuli “words” to repeated Bislama words likely to occur on a college sign.

- **(c) _blong_** is very common in Bislama signage (means “of / belonging to”), and it’s likely a short standalone word. The best match is **word 11**:  
  **11.** `<token_8><token_14>`

For the other two, we look for longer “loanword-shaped” items:

- **(a) _filosofi_** looks like “philosophy” and is fairly long; the best candidate among the long words is **word 12**.  
  **(a) filosofi → 12**

- **(b) _institiut_** (“institute”) is also long; the best candidate is **word 10**.  
  **(b) institiut → 10**

So:

- (a) **12**  
- (b) **10**  
- (c) **11**

---

### Q 6.2  (give word numbers)

Using common Bislama vocabulary expected on an educational sign (e.g., “village”, “centre”, “teach”, “technology”, “humanity”) and matching by plausible word length/pattern, the best matches are:

- **(a) village → 13**  
- **(b) centre → 8**  
- **(c) te

Calling OpenAI API: 100%|█████████████████████████| 2/2 [00:11<00:00,  5.50s/it]

### Q 6.1  (give the number of a word)

Because Bislama words on institutional signs are often recognizable borrowings from English/French, we can match by “shape” (length/pattern) and repetition:

- **(a) filosofi** → **word 15**  
- **(b) institiut** → **word 12**  
- **(c) blong** → **word 2**

---

### Q 6.2  (give the number of a word meaning …)

- **(a) village** → **word 13**
- **(b) centre** → **word 14**
- **(c) teach** → **word 4**
- **(d) technology** → **word 10**
- **(e) humanity** → **word 5**

---

### Q 6.3  Geographical area of the Pacific mentioned

**Melanesia**.


In [11]:

######## with 圖片 GPT ########

import base64
from pathlib import Path
from openai import OpenAI
from tqdm import tqdm

API_KEY = os.getenv("OPENAI_API_KEY")
if API_KEY is None:
    raise EnvironmentError("OPENAI_API_KEY is not set in the environment variables")

client = OpenAI(api_key=API_KEY)


output_file = Path("/Users/linzhenglin/Desktop/GPT_output_with_pic.txt") 
output_file.parent.mkdir(parents=True, exist_ok=True)

SEPARATOR = "\n\n\n——\n\n\n"

prompt = '''I have a dataset in an arbitrary language, but because its script is particularly difficult to format for computer input, different <token_N>, where N is the number, will represent different characters. For example, |<token_1><token_3><token_6>| can substitute for a string or sentence composed of three characters. Please use this as background knowledge for handling the following questions. 
The appearance of each token is provided in the given image.

The Avoiuli writing system is used on Pentecost Island in Vanuatu, a Pacific Island Country in the Pacific region of Melanesia. With a design inspired by traditional sand drawings, Avoiuli was developed over fourteen years by Chief Viraleo Boborenvanua, as part of a movement he leads to revitalize Melanesian culture and language.
One of the languages that Avoiuli is used to write is Bislama, one of the official languages of Vanuatu. As you will see, Bislama is a creole language – many of its words come from English, some come from French, and others from the indigenous languages of Vanuatu.
Below is a sign written in Bislama, using the Avoiuli script, on the front of a college on Pentecost Island. For the purposes of this problem, each of the words in the script below has been allotted a number:

1. <token_1><token_2><token_3>
2. <token_1><token_4><token_5><token_6><token_2>
3. <token_3><token_7><token_8><token_9>
4. <token_6><token_10><token_11><token_6><token_11><token_6><token_1><token_5><token_11>
5. <token_5><token_2><token_11><token_1><token_11><token_5><token_12><token_13><token_12><token_14>
6. <token_3><token_7><token_8><token_9>
7. <token_6><token_15><token_16><token_15><token_14>
8. <token_1><token_2><token_4><token_5>
9. <token_17><token_15><token_7><token_8><token_1><token_8><token_17><token_15>
10. <token_11><token_16><token_8><token_13><token_8><token_5><token_18><token_12><token_6>
11. <token_8><token_14>
12. <token_11><token_6><token_11><token_5><token_2><token_14><token_10><token_11><token_19>
13. <token_7><token_15><token_1><token_2>
14. <token_20><token_15><token_7><token_15><token_16>
15. <token_7><token_8><token_7><token_8><token_20><token_15><token_5><token_15>

Q 6.1 On your answer sheet, give the number of a word (any one, if there are multiple) which corresponds to these Bislama words:

(a) filosofi 

(b) institiut

(c) blong

Q 6.2 Give the numbers of a word (any one, if there are multiple) which means:

(a) village 

(b) centre 

(c) teach

(d) technology

(e) humanity 

Q 6.3 Which geographical area of the Pacific is mentioned in the text?'''

image_path = Path("/Users/linzhenglin/Desktop/avo.png")  


def image_to_data_url(path: Path) -> str:
    b64 = base64.b64encode(path.read_bytes()).decode("utf-8")
    ext = path.suffix.lstrip(".").lower() or "png"
    return f"data:image/{ext};base64,{b64}"

for i in tqdm(range(10), desc="Calling OpenAI API"): 
    content = [{"type": "text", "text": prompt}]

    if image_path is not None:
        content.append({
            "type": "image_url",
            "image_url": {
                "url": image_to_data_url(image_path)
            }
        })

    response = client.chat.completions.create(
        model="gpt-5.2",
        messages=[
            {"role": "user", "content": content}
        ],
    )

    reply = response.choices[0].message.content or ""
    print(reply)
    if output_file.stat().st_size > 0:
        reply = SEPARATOR + reply

    with output_file.open("a", encoding="utf-8") as f:
        f.write(f"[run {i}]\n{reply}")



### Q6.1  (give the word number)

To match these, use the fact that many Bislama words are close to English/French loanwords, so their Avoiuli spellings repeat in the sign:

- **(a) filosofi** → **15**  
- **(b) institiut** → **4**  
- **(c) blong** → **11**

---

### Q6.2  (give the word number)

- **(a) village** → **1**  
- **(b) centre** → **8**  
- **(c) teach** → **13**  
- **(d) technology** → **10**  
- **(e) humanity** → **12**

---

### Q6.3

The geographical area mentioned is **Melanesia**.
Done.


In [ ]:

######## prompt with VLM description GPT########


import base64
from pathlib import Path
from openai import OpenAI
from tqdm import tqdm

API_KEY = os.getenv("OPENAI_API_KEY")
if API_KEY is None:
    raise EnvironmentError("OPENAI_API_KEY is not set in the environment variables")

client = OpenAI(api_key=API_KEY)


output_file = Path("/.../GPT_output_with_gpt_des.txt")
output_file.parent.mkdir(parents=True, exist_ok=True) 

SEPARATOR = "\n\n\n——\n\n\n"

prompt = '''I have a dataset in an arbitrary language, but because its script is particularly difficult to format for computer input, different <token_N>, where N is the number, will represent different characters. For example, |<token_1><token_3><token_6>| can substitute for a string or sentence composed of three characters. Please use this as background knowledge for handling the following questions. 
The appearance descriptions for each token will be provided after the questions to facilitate solving them.

The Avoiuli writing system is used on Pentecost Island in Vanuatu, a Pacific Island Country in the Pacific region of Melanesia. With a design inspired by traditional sand drawings, Avoiuli was developed over fourteen years by Chief Viraleo Boborenvanua, as part of a movement he leads to revitalize Melanesian culture and language.
One of the languages that Avoiuli is used to write is Bislama, one of the official languages of Vanuatu. As you will see, Bislama is a creole language – many of its words come from English, some come from French, and others from the indigenous languages of Vanuatu.
Below is a sign written in Bislama, using the Avoiuli script, on the front of a college on Pentecost Island. For the purposes of this problem, each of the words in the script below has been allotted a number:

1. <token_1><token_2><token_3>
2. <token_1><token_4><token_5><token_6><token_2>
3. <token_3><token_7><token_8><token_9>
4. <token_6><token_10><token_11><token_6><token_11><token_6><token_1><token_5><token_11>
5. <token_5><token_2><token_11><token_1><token_11><token_5><token_12><token_13><token_12><token_14>
6. <token_3><token_7><token_8><token_9>
7. <token_6><token_15><token_16><token_15><token_14>
8. <token_1><token_2><token_4><token_5>
9. <token_17><token_15><token_7><token_8><token_1><token_8><token_17><token_15>
10. <token_11><token_16><token_8><token_13><token_8><token_5><token_18><token_12><token_6>
11. <token_8><token_14>
12. <token_11><token_6><token_11><token_5><token_2><token_14><token_10><token_11><token_19>
13. <token_7><token_15><token_1><token_2>
14. <token_20><token_15><token_7><token_15><token_16>
15. <token_7><token_8><token_7><token_8><token_20><token_15><token_5><token_15>

Q 6.1 On your answer sheet, give the number of a word (any one, if there are multiple) which corresponds to these Bislama words:

(a) filosofi 

(b) institiut

(c) blong

Q 6.2 Give the numbers of a word (any one, if there are multiple) which means:

(a) village 

(b) centre 

(c) teach

(d) technology

(e) humanity 

Q 6.3 Which geographical area of the Pacific is mentioned in the text?
| Token | Description |
|--------|-------------|
| Token 1 | A symmetrical inverted V-shaped symbol with two straight lines meeting at a sharp top point and slightly spreading at the base. |
| Token 2 | A symbol composed of a large oval loop on the left and a smaller circular loop nested on the right, both resting on a shared baseline. |
| Token 3 | A heart-shaped symbol formed by two rounded upper lobes that meet at a single point at the bottom. |
| Token 4 | A symbol featuring a central crossing knot with horizontal extensions on both sides and a curved line extending diagonally downward. |
| Token 5 | A three-petal vertical shape with a narrow central oval flanked by two wider curved side lobes tapering into a pointed base. |
| Token 6 | A simple inverted U-shaped arch consisting of a single smooth curved line with symmetrical sides. |
| Token 7 | A complex interwoven symbol made of multiple overlapping curved lines forming a dense, star-like knot structure. |
| Token 8 | A symmetrical symbol with two vertical oval loops positioned side by side and connected at the bottom. |
| Token 9 | A triangular frame with a pointed top enclosing two oval shapes positioned side by side at the base. |
| Token 10 | An M-shaped symbol with two tall outer arcs and inner lines crossing near the lower center. |
| Token 11 | A vertically stacked symbol with three looped lobes aligned along a central stem that intersects a horizontal baseline. |
| Token 12 | A symbol with an upper oval loop and a lower angled oval loop connected at a central crossing point on a horizontal line. |
| Token 13 | A dense, interlaced symbol formed by several overlapping curved loops creating a knot-like structure with one short extension on the right. |
| Token 14 | A symmetrical symbol with two small horizontal loops at the top connected to a long vertical oval stem that meets at a central base point. |
| Token 15 | A vertically layered three-loop structure with an upper oval, middle oval, and lower oval aligned along a central crossing baseline. |
| Token 16 | A sharply angled symbol with two diagonal lines meeting at a lower point and supporting a small inverted triangular or wedge-shaped top. |
| Token 17 | A symbol consisting of two oval loops joined across a horizontal line, with the right loop larger and the left loop partially intersected. |
| Token 18 | A rounded dome-shaped outline enclosing two small circular loops connected to a central base junction. |
| Token 19 | A symmetrical figure with a small inverted triangular top connected to a long central oval stem that meets at a single base point. |
| Token 20 | A symbol featuring a tall narrow vertical loop on the left and three small rounded loops branching from a short base stem on the right. |

'''

image_path = None  


def image_to_data_url(path: Path) -> str:
    b64 = base64.b64encode(path.read_bytes()).decode("utf-8")
    ext = path.suffix.lstrip(".").lower() or "png"
    return f"data:image/{ext};base64,{b64}"

for i in tqdm(range(10), desc="Calling OpenAI API"): 
    content = [{"type": "text", "text": prompt}]

    if image_path is not None:
        content.append({
            "type": "image_url",
            "image_url": {
                "url": image_to_data_url(image_path)
            }
        })

    response = client.chat.completions.create(
        model="gpt-5.2",
        messages=[
            {"role": "user", "content": content}
        ],
    )

    reply = response.choices[0].message.content or ""
    print(reply)
    if output_file.stat().st_size > 0:
        reply = SEPARATOR + reply

    with output_file.open("a", encoding="utf-8") as f:
        f.write(f"[run {i}]\n{reply}")



In [ ]:

######## prompt with VLM description GPT########


import base64
from pathlib import Path
from openai import OpenAI
from tqdm import tqdm

API_KEY = os.getenv("OPENAI_API_KEY")
if API_KEY is None:
    raise EnvironmentError("OPENAI_API_KEY is not set in the environment variables")

client = OpenAI(api_key=API_KEY)


output_file = Path("/.../GPT_output_VLM_des.txt")
output_file.parent.mkdir(parents=True, exist_ok=True) 

SEPARATOR = "\n\n\n——\n\n\n"

prompt = '''I have a dataset in an arbitrary language, but because its script is particularly difficult to format for computer input, different <token_N>, where N is the number, will represent different characters. For example, |<token_1><token_3><token_6>| can substitute for a string or sentence composed of three characters. Please use this as background knowledge for handling the following questions. 
The appearance descriptions for each token will be provided after the questions to facilitate solving them.

The Avoiuli writing system is used on Pentecost Island in Vanuatu, a Pacific Island Country in the Pacific region of Melanesia. With a design inspired by traditional sand drawings, Avoiuli was developed over fourteen years by Chief Viraleo Boborenvanua, as part of a movement he leads to revitalize Melanesian culture and language.
One of the languages that Avoiuli is used to write is Bislama, one of the official languages of Vanuatu. As you will see, Bislama is a creole language – many of its words come from English, some come from French, and others from the indigenous languages of Vanuatu.
Below is a sign written in Bislama, using the Avoiuli script, on the front of a college on Pentecost Island. For the purposes of this problem, each of the words in the script below has been allotted a number:

1. <token_1><token_2><token_3>
2. <token_1><token_4><token_5><token_6><token_2>
3. <token_3><token_7><token_8><token_9>
4. <token_6><token_10><token_11><token_6><token_11><token_6><token_1><token_5><token_11>
5. <token_5><token_2><token_11><token_1><token_11><token_5><token_12><token_13><token_12><token_14>
6. <token_3><token_7><token_8><token_9>
7. <token_6><token_15><token_16><token_15><token_14>
8. <token_1><token_2><token_4><token_5>
9. <token_17><token_15><token_7><token_8><token_1><token_8><token_17><token_15>
10. <token_11><token_16><token_8><token_13><token_8><token_5><token_18><token_12><token_6>
11. <token_8><token_14>
12. <token_11><token_6><token_11><token_5><token_2><token_14><token_10><token_11><token_19>
13. <token_7><token_15><token_1><token_2>
14. <token_20><token_15><token_7><token_15><token_16>
15. <token_7><token_8><token_7><token_8><token_20><token_15><token_5><token_15>

Q 6.1 On your answer sheet, give the number of a word (any one, if there are multiple) which corresponds to these Bislama words:

(a) filosofi 

(b) institiut

(c) blong

Q 6.2 Give the numbers of a word (any one, if there are multiple) which means:

(a) village 

(b) centre 

(c) teach

(d) technology

(e) humanity 

Q 6.3 Which geographical area of the Pacific is mentioned in the text?
| Token Label | VLM Token Descriptions |
|------------|------------------------|
| token_token_token_1 | The symbol is a black, symmetrical, triangular shape with a narrow peak at the top and two broad, converging lines extending downward. The lines are slightly curved inward toward the center, creating a V-shape. The black triangular shape is centered on a white background, and the edges of the triangle are blurred, giving a soft, diffused appearance. The top of the triangle is the narrowest point, while the base is the widest. The converging lines are thick at the base and taper as they approach the peak. The overall shape is reminiscent of an inverted V or a stylized arrowhead. The symmetry and convergence of the lines create a sense of depth and focus. There are no additional elements, textures, or colors; the primary focus is the black triangular shape against the white background. The strokes are smooth, and the lines are clean with no visible imperfections. The visual structure is simple yet striking due to the contrast between the black shape and the white background. |
| token_token_token_2 | The symbol is a black, monochrome graphic composed of three interconnected loops. The design features two large, overlapping circles in the lower half, which are connected by a vertical line in the center. Above these circles, there is a larger, semi-circular loop that encloses the top portion of the circles. The semi-circular loop is slightly tilted to the right. The symbol has a symmetrical structure with the two lower circles being equal in size and positioned side by side. The vertical line connecting the circles is thick and solid, creating a strong central axis. The semi-circular loop has a smooth, curved contour with a slight inward indentation near the top. The overall shape of the symbol resembles a stylized letter "P" or a simplified representation of the number "3" with additional loop elements. The lines are clean and consistent in thickness, giving the symbol a modern, minimalist appearance. There are no additional textures, gradients, or colors; the entire symbol is rendered in solid black on a white background. The design is simple yet bold, with a clear focus on geometric shapes and their interconnections. |
| token_token_token_3 | This symbol is a black, hand-drawn heart shape created using a thick, continuous line. The heart is symmetrical with a slightly elongated, pointed bottom. The line starts at the bottom center and curves upward in a smooth, continuous motion to form the top of the heart. The heart's outline is thick and bold, with a consistent width throughout. The background is white, and the heart is centered within this space. The line that forms the heart intersects at the bottom center, creating a small, triangular shadow-like shape beneath the heart. The overall style is simple and minimalist, with clean, smooth curves and no additional lines or embellishments. The heart's outline is uniform and unbroken, giving it a polished and intentional appearance. The symbol has a strong contrast between the black line and the white background, making the heart shape clearly visible and prominent. |
| token_token_token_4 | The symbol is a black, monochrome, digital drawing with a simple, minimalist design. It features a central oval shape, slightly tilted to the right, with two horizontal lines extending from the bottom of the oval, meeting at the top. The lines are evenly spaced and parallel to each other. The oval has a smooth, continuous curve with no additional lines or textures inside. The overall shape resembles a stylized representation of an eye with horizontal eyebrows. The drawing uses thick, uniform black lines with no shading or gradients. The background is entirely white, providing a stark contrast to the black lines. The symbol is symmetrically balanced with the oval in the center and the two horizontal lines evenly spaced on either side. The lines and oval are connected at the bottom, forming a triangular shape. There are no additional elements, textures, or details within the symbol. The simplicity and clarity of the lines make the symbol easily recognizable and visually distinct. |
| token_token_token_5 | The symbol consists of a black, symmetrical design on a white background. The overall shape resembles a heart, but with additional geometric elements. The top part of the symbol features two elongated, vertical lines that converge slightly at the center, forming a peak. Below these lines, there are two triangular shapes, one on each side, each with a pointed bottom and a rounded top. These triangles are connected at their bases, forming a diamond shape at the bottom center of the symbol. The lines are thick and bold, creating a strong contrast against the white background. The design has a clean, modern appearance with a focus on symmetry and geometric precision. The heart-like shape is formed by the converging vertical lines and the triangular shapes, while the diamond shape adds an additional layer of complexity. The symbol's lines are straight with some slight curvature at the points where the triangles meet. The overall structure is balanced and visually appealing, with a clear emphasis on symmetry and geometric harmony. |
| token_token_token_6 | The symbol consists of a black, arch-like shape with a smooth, continuous curve forming the top of the arch. The arch is centered vertically and horizontally in the image. The black shape is thick and uniform in width, with no visible lines or textures other than the solid black color. The arch is symmetrical, with both sides of the curve meeting at the top and extending downward to form a complete arch shape. The background is white, providing a stark contrast to the black arch. The overall shape is simple and clean, with no additional elements or decorations. The arch is the only prominent feature in the image, and its smooth, continuous curve is the primary visual element. |
| token_token_token_7 | The symbol is a black, monochrome, digital drawing that appears to be a stylized arch or frame. The arch is centered vertically and horizontally in the image. It is composed of two thick, curved lines that meet at the top to form the archway, and these lines taper slightly as they extend downward. The lines are black and have a slight gradient effect, becoming slightly darker towards the outer edges. The arch's interior is filled with a white space, creating a stark contrast with the black lines. The edges of the white space are irregular, with small, scattered black dots that give a textured, slightly worn appearance to the background. The overall shape of the arch is symmetrical, with the two curved lines being mirror images of each other. The curves are smooth and gently rounded, creating a soft, flowing appearance. There are no additional shapes, symbols, or elements within the arch itself, and the focus is entirely on the arch's structure. The style is minimalistic and modern, with a clean, simple design. The image has a slight blur effect, which adds a sense of depth and dimension to the otherwise flat design. |
| token_token_token_8 | The symbol consists of a black, monochromatic design on a white background. It features a symmetrical, abstract shape with two large, oval-like loops that are positioned vertically in the upper half of the symbol. These loops are connected at the bottom by a series of intersecting lines that form an X shape, creating a crisscross pattern. The lines are thick and evenly spaced, adding to the symmetry and balance of the design. The loops and lines are outlined in black, with no shading or gradients, giving the symbol a clean and crisp appearance. The overall shape is reminiscent of a stylized letter "E" or an abstract representation of a pair of wings. The symbol is centered on the white background, with no additional elements or text surrounding it. The design is simple yet complex, relying on geometric shapes and symmetry to create a visually striking image. The lines and loops are evenly spaced, and the symbol has a balanced and harmonious feel. |
| token_token_token_9 | The symbol is a black, monochromatic, stylized design with a symmetrical, triangular shape. The top of the symbol features a small, inverted triangle, creating a peak. Below this peak, a larger, upward-pointing triangle is formed by three intersecting, thick black lines. These lines converge at the base of the symbol, creating a triangular shape that points upward. The lower part of the symbol includes two large, circular shapes, each positioned on either side of a central vertical line. These circles are enclosed within the triangular structure created by the three intersecting lines. The vertical line runs from the top of the symbol to the bottom, intersecting the three horizontal lines at their midpoints. The entire design has a bold, thick-line appearance, with clean, sharp edges and a consistent thickness throughout the lines. The background is plain white, which contrasts with the black lines of the symbol, making the design stand out prominently. The overall shape is balanced and symmetrical, with a combination of geometric shapes and lines that create a visually striking form. The symbol has a modern, graphic style with a focus on simplicity and clarity. |
| token_token_token_10 | The symbol is a black, monochrome, digital drawing with a symmetrical structure. It consists of four thick, black, intersecting lines forming a cross-like shape. The lines are parallel and meet at a central point, creating four triangular sections. The lines are evenly spaced, with two lines running horizontally and two lines running vertically, forming a plus sign (+) shape. The intersections are precise and sharp, with no additional lines or shading. The background is white, providing a stark contrast to the black lines. The overall shape is geometric and angular, with clean, straight edges and no curves. The lines are of uniform thickness, and the symbol is centered within the frame. The style is minimalist and modern, with a focus on simplicity and clarity. There are no additional elements, textures, or colors, making the symbol easily recognizable and visually striking. |
| token_token_token_11 | The symbol is a black, monochromatic, and bold graphic character that resembles an ampersand (&). The character is composed of four main strokes: a central vertical stroke, a curved loop-like stroke from the top left, a horizontal stroke intersecting the vertical near the middle, and a smaller loop-like stroke from the bottom left. The loops are open and do not close over themselves. The character is symmetric around the vertical stroke, with the two loops on either side being mirror images. The intersections between the strokes are precise and clean, creating a balanced and structured appearance. The black lines are thick and evenly spaced, giving the character a bold and prominent appearance against a white background. |
| token_token_token_12 | The symbol consists of a black, stylized, cursive-like character with a symmetrical design. It features two main loops that intersect at the center. The upper loop is slightly larger and more open, while the lower loop is smaller and more closed. Both loops are connected by a short horizontal line that runs across the middle of the symbol. The overall shape resembles an abstract, elongated ampersand (&). The lines are thick and slightly uneven, giving the symbol a hand-drawn appearance. The upper loop is tilted slightly to the right, and the lower loop is tilted to the left, creating a sense of dynamic movement. The symbol has a monochromatic black color against a white background. The strokes are smooth but with subtle variations in thickness, adding a sense of texture. The loops and the horizontal line intersect precisely at the center, creating a balanced and harmonious composition. |
| token_token_token_13 | The symbol is a black, abstract, geometric shape composed of intersecting lines and curves. The overall shape is symmetrical and resembles a stylized, abstract butterfly or star. The figure has a central, triangular area formed by two intersecting lines that curve outward, creating a V-shape. From each of the outer points of this V-shape, diagonal lines extend outward, creating a crisscross pattern. These diagonal lines intersect with additional lines that form a loop around the central triangular area, creating a looped, open shape at the bottom. The lines are thick and black, with sharp, clean edges, and there is no shading or texture. The background is white, making the black lines stand out prominently. The entire symbol is centered in the image, with a balanced and symmetrical structure. |
| token_token_token_14 | The symbol is a black, monochromatic, abstract design on a white background. It consists of two large, oval-shaped loops that intersect at the top center, forming a symmetrical, butterfly-like structure. Below the loops, there are two straight lines that converge at the bottom center, creating a V-shape. The loops and straight lines are connected at their points of intersection, forming a cohesive, balanced shape. The design uses clean, continuous lines with no textures or gradients. The loops have a slightly thicker outline than the straight lines. The symbol is centered within the white background and has a minimalist, modern aesthetic focused on symmetry and geometric forms. |
| token_token_token_15 | The symbol is a black, monochromatic, typographic character with a distinct, bold design. It features three horizontal lines intersecting with two vertical lines, creating a symmetrical, cross-like structure. The horizontal lines are thick and evenly spaced, while the vertical lines are thinner and positioned at the center of each horizontal line. The intersections form oval shapes above and below the center. The overall shape resembles an ampersand (&) with additional horizontal elements. The lines are precise and consistent in thickness, giving the symbol a clean, geometric appearance with a slightly three-dimensional feel. |
| token_token_token_16 | The symbol is a black, monochromatic, highly stylized design rendered digitally. It combines curved and straight lines in a balanced, symmetrical composition. A central horizontal line runs across the middle, with two semi-circular loops above and below it. These loops are thick, smooth, and slightly three-dimensional in appearance. A diagonal vertical line intersects the horizontal line, positioned centrally between the loops. The entire symbol is outlined in thick black with no fill, standing out clearly against the white background. The style is modern, geometric, and minimalist, with an emphasis on clarity and structure. |
| token_token_token_17 | The symbol consists of two overlapping semi-circular shapes that intersect at their centers. The top semi-circle is larger and slightly offset to the right, while the smaller one sits to the left. Both have thick black outlines with a slightly jagged, hand-drawn edge. A central horizontal line runs through the middle of the overlapping shapes, also thick and uneven. The background is white, creating strong contrast. The overall shape is symmetrical, with wavy lines adding a dynamic, hand-rendered quality. |
| token_token_token_18 | The symbol is a monochromatic black-and-white abstract line drawing. It features a large circular head-like shape with a smooth outline and two darker circular features positioned symmetrically inside, resembling eyes or ears. From the bottom of the head, a straight line extends downward and splits into two smaller lines forming an X shape. The lines are thin, evenly spaced, and clean. The overall design is minimalistic and symmetrical, with a strong focus on simple geometric forms and balance. |
| token_token_token_19 | The symbol is a monochrome geometric design with a symmetrical structure. A central vertical oval is flanked by two inward-pointing triangles whose bases touch the oval. At the top, two horizontal lines form a rectangular shape that connects with the triangles. The lines are thick, evenly spaced, and sharply defined. The design is sleek, modern, and minimalist, emphasizing geometric precision and symmetry against a white background. |
| token_token_token_20 | The symbol consists of three primary black shapes arranged vertically. At the top is a large, elongated curved loop with a thick, slightly irregular outline. Beneath it are two small circular face-like shapes with single white eyes and subtle curved mouths. At the bottom is a large solid black triangular shape with a rounded base and pointed top. Short curved lines connect the faces to the triangle. The composition is symmetrical and vertically aligned, using thick black lines and strong contrast against a white background, combining abstract and figurative elements into a cohesive form. |
'''

image_path = None


def image_to_data_url(path: Path) -> str:
    b64 = base64.b64encode(path.read_bytes()).decode("utf-8")
    ext = path.suffix.lstrip(".").lower() or "png"
    return f"data:image/{ext};base64,{b64}"

for i in tqdm(range(10), desc="Calling OpenAI API"): 
    content = [{"type": "text", "text": prompt}]

    if image_path is not None:
        content.append({
            "type": "image_url",
            "image_url": {
                "url": image_to_data_url(image_path)
            }
        })

    response = client.chat.completions.create(
        model="gpt-5.2",
        messages=[
            {"role": "user", "content": content}
        ],
    )

    reply = response.choices[0].message.content or ""
    print(reply)
    if output_file.stat().st_size > 0:
        reply = SEPARATOR + reply

    with output_file.open("a", encoding="utf-8") as f:
        f.write(f"[run {i}]\n{reply}")

